In [13]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file CSV (thêm tên cột nếu thiếu)
file_path = "patient_heart_rate.csv"
df = pd.read_csv(file_path, header=None, encoding='utf-8', on_bad_lines='skip')

# Đặt tên cột nếu thiếu tiêu đề
columns = ["ID", "Name", "Age", "Weight", "HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
df.columns = columns[:len(df.columns)]





In [14]:
# Xử lý vấn đề 2: Tách cột "Name" thành "Firstname" và "Lastname"
# (Đã xử lý trước đó, không cần thực hiện lại)

# Xử lý vấn đề 3: Chuẩn hóa đơn vị đo lường của cột Weight
def convert_weight(value):
    if pd.isnull(value):
        return np.nan
    try:
        if "kg" in str(value):
            return float(str(value).replace("kg", ""))
        elif "lbs" in str(value):
            return round(float(str(value).replace("lbs", "")) * 0.453592, 2)  # Chuyển lbs -> kg
        else:
            # Attempt to convert directly to float if no unit is present
            return float(value)
    except ValueError:
        return np.nan  # Return NaN for invalid strings

df['Weight'] = df['Weight'].apply(convert_weight)

# Xử lý vấn đề 4: Xóa các dòng trống hoàn toàn
df.dropna(how='all', inplace=True)

# Xử lý vấn đề 5: Xóa các dòng trùng lặp hoàn toàn
df.drop_duplicates(inplace=True)

In [15]:
# Xử lý vấn đề 6: Thay thế ký tự không phải ASCII bằng "warning"
df = df.applymap(lambda x: x if isinstance(x, (int, float)) or x.isascii() else "warning")

# Xử lý vấn đề 7: Điền giá trị thiếu
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Weight'].fillna(df['Weight'].mean(), inplace=True)
df.dropna(subset=['Age', 'Weight'], how='all', inplace=True)

# Xử lý vấn đề 8: Tách dữ liệu trong header
if df.columns[0].startswith("m") or df.columns[0].startswith("f"):
    df['Sex'] = df.columns[0][0]
    df['Time'] = df.columns[0][1:]
    df['PulseRate'] = df.iloc[:, 0]
    df.drop(columns=[df.columns[0]], inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_3200\2693394813.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x if isinstance(x, (int, float)) or x.isascii() else "warning")
C:\Users\capta\AppData\Local\Temp\ipykernel_3200\2693394813.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\capta\AppData\Local\Temp\ipykernel_3200\2693394813.py:6: FutureWarning: A value is trying to be set on a copy of a DataFr

In [19]:
def fill_missing_bp(row, cols):
    for col in cols:
        if pd.isna(row[col]):
            prev_vals = [row[c] for c in cols[:cols.index(col)] if not pd.isna(row[c])]
            next_vals = [row[c] for c in cols[cols.index(col)+1:] if not pd.isna(row[c])]
            
            if prev_vals and next_vals:
                row[col] = (prev_vals[-1] + next_vals[0]) / 2  # Trung bình liền trước và liền sau
            elif len(prev_vals) >= 2:
                row[col] = np.mean(prev_vals[-2:])  # Trung bình 2 giá liền trước
            elif len(next_vals) >= 2:
                row[col] = np.mean(next_vals[:2])  # Trung bình 2 giá liền sau
            elif prev_vals or next_vals:
                row[col] = np.mean(prev_vals + next_vals)  # Trung bình các giá trị huyết áp của người đó
            elif 'Sex' in df.columns:
                sex_mean = df[df['Sex'] == row['Sex']][col].mean()
                if not np.isnan(sex_mean):
                    row[col] = sex_mean  # Trung bình của nhóm giới tính
                else:
                    row[col] = df[col].mean()  # Trung bình của toàn bộ dữ liệu
    return row

hr_columns = ["HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
# Convert HR columns to numeric, replacing invalid entries with NaN
for col in hr_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.apply(lambda row: fill_missing_bp(row, hr_columns), axis=1)


# Ensure 'Firstname', 'Lastname', and 'Sex' columns exist
if 'Name' in df.columns:
    df[['Firstname', 'Lastname']] = df['Name'].str.split(' ', n=1, expand=True)
else:
    df['Firstname'] = np.nan
    df['Lastname'] = np.nan

if 'Sex' not in df.columns:
    df['Sex'] = np.nan

columns_order = ["ID", "Firstname", "Lastname", "Age", "Weight", "Sex", "HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
df = df[columns_order]
df.reset_index(drop=True, inplace=True)

# Xuất dữ liệu đã xử lý
output_path = "patient_heart_rate_clean.csv"
df.to_csv(output_path, index=False)
print("Dữ liệu đã được xử lý và lưu vào:", output_path)

Dữ liệu đã được xử lý và lưu vào: patient_heart_rate_clean.csv
